# 1. Information about the submission

## 1.1 Name and number of the assignment 

Assignment 1 Text Classification

## 1.2 Student name

Prateek Rajput

## 1.3 Codalab user ID

pkrajput7

## 1.4 Additional comments

The accuracy in the report for my final implementation is missing because the model took too more time to train than anticipated, I will put both of the f1 scores in comments. Please pardon me for this. 

# 2. Technical Report

*Use Section 2 to describe results of your experiments as you would do writing a paper about your results. DO NOT insert code in this part. Only insert plots and tables summarizing results as needed. Use formulas if needed do described your methodology. The code is provided in Section 3.*

## 2.1 Methodology 

1) Tokenizing and embeddings: I used the basic keras text tokenizer with padding. for embeddings I tried to use transformer models (BERT) and also ELMo to improve the result over the seminar baseline but the main problem was including it in the keras framework, I defined the special class for the Embedding layer but it kept giving an error about some layer superposition definition so finally I went with the native keras encoder \\

2) size of the embedding vector and drop out: I changed the size of the embedding vector to 150 to capture some relational dynamics that the bi directional LSTM will capture and improve the accuracy of my implementation \\

3) Bi directional LSTM and drop out: I tried bi directional LSTM instead of the unidirectional architecture and played around with the drop out rate for regularising because a super high accuracy on the training set was shuoting overfitted model to me ) I also tried to use the GRU cell instead of LSTMs but that did not work out quite well \\

4) Spacial drop out: I eliminated the spacial drop out in the final implementation, just because it achieved the highest accuracy for me.

Bidirectional LSTM:

![image](https://analyticsindiamag.com/complete-guide-to-bidirectional-lstm-with-python-codes/)


## 2.2 Discussion of results

My initial implementation with LSTM RNN network was above baseline (results are indicated below) but my final implementation achieved a higher result as can be seen in the table

Method | F1 stance detection | F1 stance classification
--- | --- | ---
Baseline | 0.4180 | 0.4355
RNN (uni directional LSTM memory cell) | 0.4453 | 0.4564
RNN (bi directional LSTM memory cell) |  | 

The LSTM network took 1.5 hrs to train while the bi Directional LSTM network took 3.5 hrs to train on 2 GPUs 



# 3. Code

*Enter here all code used to produce your results submitted to Codalab. Add some comments and subsections to navigate though your solution.*

*In this part you are expected to develop yourself a solution of the task and provide a reproducible code:*
- *Using Python 3;*
- *Contains code for installation of all dependencies;*
- *Contains code for downloading of all the datasets used*;
- *Contains the code for reproducing your results (in other words, if a tester downloads your notebook she should be able to run cell-by-cell the code and obtain your experimental results as described in the methodology section)*.


*As a result, you code will be graded according to these criteria:*
- ***Readability**: your code should be well-structured preferably with indicated parts of your approach (Preprocessing, Model training, Evaluation, etc.).*
- ***Reproducibility**: your code should be reproduced without any mistakes with “Run all” mode (obtaining experimental part).*


## 3.1 Requirements

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
#stopwords = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
#import plotly.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
from nltk.corpus import stopwords
from keras.layers import Bidirectional

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 3.2 Download the data

In [2]:
!wget -O train.tsv https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/train.tsv
!wget -O val_empty.tsv https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/val_empty.tsv
!wget -O test-no_labels.tsv https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/test-no_labels.tsv
# if some needed file is not in the public domain use google drive or other free hosting to make them available

--2022-04-17 19:08:50--  https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1539551 (1.5M) [text/plain]
Saving to: ‘train.tsv’

train.tsv           100%[===================>]   1.47M  --.-KB/s    in 0.07s   

2022-04-17 19:08:50 (21.1 MB/s) - ‘train.tsv’ saved [1539551/1539551]

--2022-04-17 19:08:50--  https://raw.githubusercontent.com/dialogue-evaluation/RuArg/main/data/val_empty.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 308303 (301K) [text/pl

In [3]:
train_df = pd.read_csv("train.tsv", sep="\t")
val_df = pd.read_csv("val_empty.tsv", sep="\t")
test_df = pd.read_csv("test-no_labels.tsv", sep="\t")

In [4]:
train_df.head()
len(test_df)
for elem in train_df.text[:10]:
  print (elem)

,text_id,text,masks_stance,masks_argument,quarantine_stance,quarantine_argument,vaccines_stance,vaccines_argument
0,17024,"[USER], согласно предписаниям Роспотребнадзора...",-1,-1,1,1,-1,-1
1,17025,О несоблюдении карантинных мер контактными лиц...,-1,-1,1,1,-1,-1
2,17027,"[USER], читайте больше книжек на карантине, мо...",-1,-1,1,1,-1,-1
3,17030,Иди почитай инсту наших городских пабликов где...,-1,-1,1,1,-1,-1
4,17031,"Все контактные лица, которых они обозначили, о...",-1,-1,1,1,-1,-1


1402

[USER], согласно предписаниям Роспотребнадзора, все транзитные пассажиры, следующие через Москву, должны пройти 14-дневный карантин у себя по месту жительства.
О несоблюдении карантинных мер контактными лицами можно сообщить на на горячую линию Роспотребнадзора - 8-800-100-0004 (можно позвонить бесплатно из любого населенного пункта России в рабочие дни с 10-00 до 17-00 (время московское), перерыв с 12-00 до 12-45) или в Единый консультационный центр Роспотребнадзора - 8-800-555-49-43.
[USER], читайте больше книжек на карантине, может мозг реанимируете
Иди почитай инсту наших городских пабликов где каждый 10ый пишет что у них знакомый откуда то вернулся и плевал на карантин, у нас никто ниче не раздает, раздавать нечего в аптеках нет ничего)
Все контактные лица, которых они обозначили, отправлены на 14-дневный карантин, им сделают тестирование.
[USER], многие находятся на домашнем карантине
Если бы сразу запретили перелеты и ввели карантин, то никто бы не полетел никуда.
[USER], надеем

## 3.3 Preprocessing 

In [5]:
!pip install tensorflow==2.2 
!pip install keras==2.4.3

In [6]:
import tensorflow_hub as hub
from keras import backend as K
import tensorflow as tf
from keras.engine import Layer

In [7]:
import tensorflow.compat.v1 as tf

class ModuleLayer(tf.keras.layers.Layer):

  @staticmethod
  def __init_session__(session):
    tf.keras.backend.set_session(session)




In [8]:
class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

        self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
        super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

## 3.4 My method of text processing

In [9]:
pip install numpy==1.19.5

In [ ]:
df1 = train_df[['masks_stance', 'masks_argument', 'quarantine_stance', 'quarantine_argument', 'vaccines_stance', 'vaccines_argument']]
for col in df1:
  # The maximum number of words to be used. (most frequent)
  MAX_NB_WORDS = 50000
  # Max number of words in each sequence
  MAX_SEQUENCE_LENGTH = 250
  EMBEDDING_DIM = 150

  tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
  tokenizer.fit_on_texts(train_df['text'].values)
  word_index = tokenizer.word_index

  X = tokenizer.texts_to_sequences(train_df['text'].values)
  X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
  print(X.shape[1])

  Y = pd.get_dummies(train_df[col]).values 

  X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.01, random_state = 42)

  model = Sequential()
  model.add(ModuleLayer())
  # model.add(ElmoEmbeddingLayer())
  model.add(Embedding(20018, EMBEDDING_DIM, input_length=X.shape[1]))
  model.add(SpatialDropout1D(0.5))
  model.add(Bidirectional(LSTM(150)))
  # model.add(LSTM(100, dropout=0.5, recurrent_dropout=0.5))
  model.add(Dense(4, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  epochs = 10
  batch_size = 64


  print (X_train.shape)
  print (Y_train.shape)
  history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size)

  x = tokenizer.texts_to_sequences(test_df['text'].values)
  x = pad_sequences(x, maxlen=MAX_SEQUENCE_LENGTH)

  predict_proba = model.predict(x)
  predict = []
  for prob in predict_proba:
    predict.append(np.argmax(prob))

  test_df[col] = predict
  test_df[col] -= 1

250
(6649, 250)
(6649, 4)
Epoch 1/10
104/104 [==============================] - 131s 1s/step - loss: 0.8599 - accuracy: 0.6609
Epoch 2/10
104/104 [==============================] - 130s 1s/step - loss: 0.4622 - accuracy: 0.8057
Epoch 3/10
104/104 [==============================] - 130s 1s/step - loss: 0.3939 - accuracy: 0.8288
Epoch 4/10
104/104 [==============================] - 129s 1s/step - loss: 0.2685 - accuracy: 0.8949
Epoch 5/10
104/104 [==============================] - 129s 1s/step - loss: 0.1543 - accuracy: 0.9475
Epoch 6/10
104/104 [==============================] - 129s 1s/step - loss: 0.0858 - accuracy: 0.9725
Epoch 7/10
104/104 [==============================] - 127s 1s/step - loss: 0.0586 - accuracy: 0.9824
Epoch 8/10
104/104 [==============================] - 128s 1s/step - loss: 0.0359 - accuracy: 0.9896
Epoch 9/10
104/104 [==============================] - 128s 1s/step - loss: 0.0224 - accuracy: 0.9938
Epoch 10/10
104/104 [==============================] - 129s 1s/st

In [ ]:
test_df.to_csv('output7.tsv', index = False, sep='\t')

In [ ]:
!zip output7.zip output7.tsv